In [120]:
# import torch and other libraries
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
from PIL import Image
from IPython.display import display # to display images

print(torch.__version__)

0.3.1


In [141]:
# other imports
import time
import numpy as np
import pickle

In [122]:
# Load the pretrained model
resnet_model = models.resnet18(pretrained=True)
alexnet_model = models.alexnet(pretrained=True)
# Use the model object to select the desired layer
resnet_layer = resnet_model._modules.get('avgpool')
alexnet_layer = alexnet_model._modules.get('classifier')

In [123]:
resnet_model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=(1, 1), ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (b

In [124]:
# Set model to evaluation mode
resnet_model.eval()
alexnet_model.eval()

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace)
    (5): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Dropout(p=0.5)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
    (2): ReLU(inplace)
    (3): Dropout(p=0.5)
    (4): Linear(in_features=4096,

In [125]:
# image scaler to 224 x 224 pixels
scaler = transforms.Scale((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
to_tensor = transforms.ToTensor()

In [126]:
def get_vector(image_name, model):
    # 1. Load the image with Pillow library
    img = Image.open(image_name)
    # 2. Create a PyTorch Variable with the transformed image
    t_img = Variable(normalize(to_tensor(scaler(img))).unsqueeze(0))
    if model == 'resnet':
        # 3. Create a vector of zeros that will hold our feature vector
        #    The 'avgpool' layer has an output size of 512
        my_embedding = torch.zeros(512)
        # 4. Define a function that will copy the output of a layer
        def copy_data(m, i, o):
            my_embedding.copy_(o.data)
        # 5. Attach that function to our selected layer
        h = resnet_layer.register_forward_hook(copy_data)
        # 6. Run the model on our transformed image
        resnet_model(t_img)
        h.remove()
    elif model == 'alexnet':
        print('using alexnet...')
        # 3. Create a vector of zeros that will hold our feature vector
        #    The 'classifier' layer has an output size of 1000
        my_embedding = torch.zeros(1000)
        def copy_data(m, i, o):
            my_embedding.copy_(o.data)
        # 5. Attach that function to our selected layer
        h = alexnet_layer.register_forward_hook(copy_data)
        # 6. Run the model on our transformed image
        alexnet_model(t_img)
        h.remove()
    # 8. Return the feature vector
    return my_embedding

In [127]:
DATA_PATH = 'data/images/'
img1 = DATA_PATH + 'golden1.jpg'
# img2 = DATA_PATH + 'golden2.jpg'
img2 = DATA_PATH + 'cat1.jpg'

In [128]:
# get feature vectors from resnet18
n_trials = 10
img1_resnet_times = []
img2_resnet_times = []

# get feature vectors n_trial times each and average
for i in range(n_trials):
    # image 1
    t_start_img1 = time.time()
    img1_vector = get_vector(img1, 'resnet')
    img1_resnet_times.append(time.time() - t_start_img1)
    # image 2
    t_start_img2 = time.time()
    img2_vector = get_vector(img2, 'resnet')
    img2_resnet_times.append(time.time() - t_start_img2)

# output results
t_mean_img1_resnet = round(np.mean(img1_resnet_times)/n_trials, 3)
t_mean_img2_resnet = round(np.mean(img2_resnet_times)/n_trials, 3)
print('Number of Trials: {}'.format(n_trials))
print('Image 1 Feature Vector Genration Time: {}s'.format(t_mean_img1_resnet))
print('Image 2 Feature Vector Genration Time: {}s'.format(t_mean_img2_resnet))

n_frames = 100
n_corpus = 3e6
t_corpus = round(t_mean_img1_resnet*n_frames*n_corpus/3600, 3)
print('\nCorpus Size={}, Frames per Video={}'.format(int(n_corpus), n_frames))
print('Corpus Processing Time: {} hrs, or {} days'.format(t_corpus, round(t_corpus/24, 3)))

Number of Trials: 10
Image 1 Feature Vector Genration Time: 0.011s
Image 2 Feature Vector Genration Time: 0.009s

Corpus Size=3000000, Frames per Video=100
Corpus Processing Time: 916.667 hrs, or 38.194 days


In [129]:
# Using PyTorch Cosine Similarity
cos = nn.CosineSimilarity(dim=1, eps=1e-6)
cos_sim = cos(img1_vector.unsqueeze(0),
              img2_vector.unsqueeze(0))
print('\nCosine similarity: {0}\n'.format(cos_sim))


Cosine similarity: 
 0.5390
[torch.FloatTensor of size 1]




Resnet 18 experiments show that feature vector image generation takes aobut 1/100th of a second. This means over 45 days to process 3 million videos with 100 sampled images each. 

In [11]:
# get feature vectors from alexnet
# img1_vector_a = get_vector(img1, 'alexnet')
# img2_vector_a = get_vector(img2, 'alexnet')

In [12]:
img1_vec = np.array(img1_vector)
img2_vec = np.array(img2_vector)
vec = np.array([img1_vec, img2_vec])
print(vec.shape)

(2, 512)


## Frames extraction from videos

In [111]:
import os
import argparse
import FFMPEGFrames
import shutil
import importlib
importlib.reload(FFMPEGFrames)

<module 'FFMPEGFrames' from '/mnt/c/Users/RyanBae/Dropbox/uw_data_science/fall_2018/data590_capstone1/VideoSimilarity/FFMPEGFrames.py'>

In [139]:
videos_path = 'data/videos'
delete = True
features = {}

# loop through all the videos
i = 0
for path, subdirs, files in os.walk(videos_path):
    for name in files:
        if i < 2:
            # extract frames from videos
            video_length = int(float(FFProbe(os.path.join(path, name)).video[0].duration)) + 1
            n_frames = 10
            fps = n_frames/video_length
            video_path = os.path.join(path, name)
            print(os.path.join(path, name) + ' ' + str(video_length) + 's ' + '(' + str(fps) + ')')
            f = FFMPEGFrames.FFMPEGFrames("data/video_frames/")
            f.extract_frames(os.path.join(path, name), fps)
            i += 1
            # get feature vectors for each frame image
            frames_path = f.full_output
            frames = os.listdir(frames_path)
            model = 'resnet'
            features[video_path] = [get_vector(os.path.join(frames_path, frame), model) for frame in frames]
            # delete rest of the files after extracting image features
            if delete:
                shutil.rmtree(f.full_output)

data/videos/animals/0.mp4 1s (10.0)
data/videos/animals/1.mp4 7s (1.4285714285714286)


[
  0.0000
  0.3110
  0.8582
  0.3950
  0.0368
  0.2640
  0.0767
  3.5520
  1.0235
  0.4385
  0.0012
  0.0186
  0.2356
  0.4036
  0.3292
  0.2740
  0.0000
  0.0005
  0.0285
  0.1234
  0.4049
  0.2204
  0.0000
  0.0314
  0.0000
  0.0024
  0.2200
  0.4574
  0.0001
  1.1677
  0.0520
  0.6322
  0.3369
  1.6916
  0.0372
  0.8176
  0.0000
  0.0000
  0.6562
  0.4905
  0.1491
  0.7550
  1.6855
  0.0074
  1.3966
  0.0000
  0.7390
  0.0000
  0.4970
  0.0074
  0.0037
  0.0778
  0.0414
  0.8775
  0.4118
  1.9250
  6.4772
  1.2403
  0.5329
  2.5796
  0.0372
  0.2381
  0.0226
  0.0763
  0.3601
  0.6849
  1.1473
  0.0000
  0.0000
  0.0000
  0.3765
  0.0061
  0.5981
  0.2050
  1.3076
  0.0000
  0.0835
  0.1009
  0.0808
  0.5708
  0.0000
  3.6382
  0.0017
  0.5387
  0.1656
  1.0769
  1.3745
  0.9549
  0.3067
  0.0090
  0.1819
  0.3536
  0.0000
  0.2075
  0.1509
  1.2253
  1.4250
  0.0000
  2.3475
  0.7885
  0.1056
  2.5858
  0.1156
  0.0020
  0.7199
  0.5632
  1.3766
  0.0238
  0.3441
  0.2831
  0.8641